# Comparison Analysis Between Different Frameworks
---

Compare between popular big data storage layers and raw file formats in terms of performance and usage. \
Testing is done by comparing performance on high and low cardinality columns to simulate real big data environments.

**Compared:**
- `Delta Lake`: Using partitioning and Z-order sort
- `Apache Iceberg`: Using partitioning, bucketing and Z-order sort
- `Hive Metastore (High Cardinality)`: Spark default hive metastore table with bucketing
- `Hive Metastore (Low Cardinality)`: Spark default hive metastore table with partitioning
- `Parquet`: Raw Parquet
- `CSV`: Raw CSV single file

### Spark Session Setup
---
Create a `SparkSession` which can utilize both Delta and Iceberg storage layers
- To keep the test environment constant, have fixed driver and executor memory at 2GB
- Local hive metastore has been set under `data/spark-warehouse` as default and will store all data and metadata there

In [30]:
import time
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from delta.tables import DeltaTable
from subprocess import check_output
from pyspark.sql.window import Window
from spark_config import create_spark_session

In [31]:
# Create custom Spark session
spark = create_spark_session(
    app="ComparisonApp",
    warehouse_location="../data/",
    deltadb="delta_db",
    icebergdb="iceberg_db"
)

In [3]:
# Confirm warehouse path is set to where we want it to be
spark.sql("DESCRIBE DATABASE EXTENDED default").show(truncate=False)

+--------------+-----------------------------------------------------------------+
|info_name     |info_value                                                       |
+--------------+-----------------------------------------------------------------+
|Catalog Name  |spark_catalog                                                    |
|Namespace Name|default                                                          |
|Comment       |default database                                                 |
|Location      |file:/Users/wonseokchoi/Documents/Projects/data_quality_mini/data|
|Owner         |wonseokchoi                                                      |
|Properties    |                                                                 |
+--------------+-----------------------------------------------------------------+



### The Data
---
- **Main Table**: Main dataset with designated number of rows generated
- **High Cardinality**: Secondary table with distinct ID column
- **Low Cardinality**: Secondary table with low cardinality link to Main Table

In [4]:
# Designated number of rows to generate
num_rows = 100_000_000

# Main Table Dataframe
main_df = spark.range(num_rows).selectExpr(
    "id as user_id", 
    "(id % 1000) as product_id", 
    "cast(rand() * 100 as int) as price", 
    "(id % 365) as day_of_year"
).withColumn(
    "product_name", 
    F.expr("CASE WHEN (product_id % 5) = 0 THEN 'Laptop' " +
            "WHEN (product_id % 5) = 1 THEN 'Mobile' " +
            "WHEN (product_id % 5) = 2 THEN 'Tablet' " +
            "WHEN (product_id % 5) = 3 THEN 'Headphones' " +
            "ELSE 'Accessories' END")
).withColumn(
    "transaction_type", 
    F.expr("CASE WHEN (price < 50) THEN 'Low Value' " +
            "WHEN (price >= 50 AND price < 80) THEN 'Medium Value' " +
            "ELSE 'High Value' END")
)

# High Cardinality Table Dataframe
high_cardinality_df = spark.range(num_rows).selectExpr(
    "id as transaction_id",
    "cast(rand() * 500 as int) as store_id",
    "cast(rand() * 100 as int) as discount"
).withColumn(
    "transaction_type", 
    F.expr("CASE WHEN (discount < 20) THEN 'Low Discount' " +
            "WHEN (discount >= 20 AND discount < 50) THEN 'Medium Discount' " +
            "ELSE 'High Discount' END")
)

# Low Cardinality Table Dataframe
low_cardinality_df = spark.range(num_rows // 100).selectExpr(
    "id as store_id",
    "(id % 3) as store_region"
).withColumn(
    "store_type", 
    F.expr("CASE WHEN (store_region = 0) THEN 'Urban' " +
            "WHEN (store_region = 1) THEN 'Suburban' " +
            "ELSE 'Rural' END")
)

main_df.printSchema()
high_cardinality_df.printSchema()
low_cardinality_df.printSchema()

root
 |-- user_id: long (nullable = false)
 |-- product_id: long (nullable = true)
 |-- price: integer (nullable = true)
 |-- day_of_year: long (nullable = true)
 |-- product_name: string (nullable = false)
 |-- transaction_type: string (nullable = false)

root
 |-- transaction_id: long (nullable = false)
 |-- store_id: integer (nullable = true)
 |-- discount: integer (nullable = true)
 |-- transaction_type: string (nullable = false)

root
 |-- store_id: long (nullable = false)
 |-- store_region: long (nullable = true)
 |-- store_type: string (nullable = false)



In [5]:
# Function to execute SQL query and return DataFrame
def run_sql(query: str, show: bool=False) -> DataFrame:
    """
    Run Spark.sql() method with option to return DataFrame head using .show() method.
    Outputs time elapsed to run query.
    :param query: SQL query to run
    :param show: Boolean to indicate if the DataFrame should be shown
    :return: None or Spark DataFrame Head
    """
    start_time = time.time()
    result = spark.sql(query)
    
    if show:
        result.show(n=5, truncate=False)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time elapsed: {elapsed_time} seconds")
    
    return result

### Setup
---

#### 1. Delta Lake Setup

In [6]:
q1_delta = """
    CREATE TABLE IF NOT EXISTS delta_db.main_table (
        user_id BIGINT,
        product_id BIGINT,
        price INT,
        day_of_year LONG,
        product_name STRING,
        transaction_type STRING
    ) USING delta
    PARTITIONED BY (product_name, day_of_year);
"""

run_sql(query=q1_delta, show=False)

Time elapsed: 4.613253831863403 seconds


DataFrame[]

#### 2. Iceberg Setup

In [7]:
q1_iceberg = """
    CREATE TABLE IF NOT EXISTS iceberg_db.main_table (
        user_id bigint,
        product_id bigint,
        price int,
        day_of_year long,
        product_name string,
        transaction_type string
    ) USING iceberg
    PARTITIONED BY (bucket(100, product_id)); 
"""

run_sql(query=q1_iceberg, show=False)

Time elapsed: 0.42108607292175293 seconds


DataFrame[]

#### 3. Metastore Table Setup

In [8]:
q0_default = "CREATE DATABASE IF NOT EXISTS spark_catalog.default_db"

q1_default = """
    CREATE TABLE IF NOT EXISTS default_db.default_bucketed_table (
        user_id bigint,
        product_id bigint,
        price int,
        day_of_year long,
        product_name string,
        transaction_type string
    ) USING parquet
    CLUSTERED BY (product_id) INTO 100 BUCKETS;
"""

q2_default = """
    CREATE TABLE IF NOT EXISTS default_db.default_partitioned_table (
        user_id bigint,
        product_id bigint,
        price int,
        day_of_year long,
        product_name string,
        transaction_type string
    ) USING parquet
    PARTITIONED BY (day_of_year);
"""

run_sql(query=q0_default, show=False)
run_sql(query=q1_default, show=False)
run_sql(query=q2_default, show=False)

Time elapsed: 0.013006210327148438 seconds
Time elapsed: 0.07683086395263672 seconds
Time elapsed: 0.01677989959716797 seconds


DataFrame[]

### Initial Write Comparison
---

#### 0. Raw CSV

In [9]:
start_time = time.time()
main_df.write \
    .csv("../data/csv/main_table", mode="overwrite")
print("Default Raw CSV Write: --- %s seconds ---" % (time.time() - start_time))

Default Raw CSV Write: --- 16.198310136795044 seconds ---


#### 1. Raw Parquet

In [10]:
start_time = time.time()
main_df.write \
    .parquet("../data/parquet/main_table", mode="overwrite")
print("Default Parquet Write: --- %s seconds ---" % (time.time() - start_time))

Default Parquet Write: --- 16.16503095626831 seconds ---


#### 2. Hive Metastore Tables

In [11]:
start_time = time.time()
main_df.write \
    .bucketBy(100, "product_id") \
    .sortBy("user_id") \
    .mode("overwrite") \
    .saveAsTable("default_db.default_bucketed_table")
print("Default Bucketing Write: --- %s seconds ---" % (time.time() - start_time))

Default Bucketing Write: --- 205.63260102272034 seconds ---


In [12]:
start_time = time.time()
main_df.write \
    .partitionBy("day_of_year") \
    .mode("overwrite") \
    .saveAsTable("default_db.default_partitioned_table")
print("Default Partitioning Write: --- %s seconds ---" % (time.time() - start_time))

Default Partitioning Write: --- 59.914121866226196 seconds ---


#### 3. Delta Lake

In [13]:
start_time = time.time()
main_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("delta_db.main_table")
print("Default Delta Write: --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
# Optimize Z-Ordering (This time use pythonic API)
deltaTable = DeltaTable.forName(spark, "delta_db.main_table")
optimizeBuilder = deltaTable.optimize()
optimizeBuilder.executeZOrderBy(["user_id", "product_id"])
print("Default Delta Optimize: --- %s seconds ---" % (time.time() - start_time))

Default Delta Write: --- 97.1530191898346 seconds ---


Default Delta Optimize: --- 199.96773290634155 seconds ---


#### 4. Iceberg

In [14]:
start_time = time.time()
main_df.write \
    .format("iceberg") \
    .mode("overwrite") \
    .saveAsTable("iceberg_db.main_table")
print("Default Iceberg Write: --- %s seconds ---" % (time.time() - start_time))

iceberg_zorder = """
CALL iceberg_db.system.rewrite_data_files(
    table => 'iceberg_db.main_table', strategy => 'sort', 
    sort_order => 'zorder(user_id, product_id)'
)
"""
start_time = time.time()
run_sql(
    query = iceberg_zorder, show=False
)
print("Default Iceberg Optimize: --- %s seconds ---" % (time.time() - start_time))

Default Iceberg Write: --- 20.474295139312744 seconds ---


Time elapsed: 213.2428011894226 seconds
Default Iceberg Optimize: --- 213.2480080127716 seconds ---


### Compression Comparison
---

In [32]:
# List of paths to check
paths = [
    "../data/csv/main_table",
    "../data/parquet/main_table",
    "../data/default_db.db/default_bucketed_table",
    "../data/default_db.db/default_partitioned_table",
    "../data/delta_db.db/main_table",
    "../data/iceberg_db/main_table"
]

# Output formatted sizes
for path in paths:
    try:
        size = check_output(["du", "-sh", path]).decode("utf-8").split()[0]
        label = "/".join(path.split("/")[-2:])
        print(f"{label}: {size}")
    except Exception as e:
        print(f"Failed to get size for {path}: {e}")

csv/main_table: 3.8G
parquet/main_table: 645M
default_db.db/default_bucketed_table: 598M
default_db.db/default_partitioned_table: 621M
delta_db.db/main_table: 1.2G
iceberg_db/main_table: 470M


### High Cardinality Join Comparison
---

#### Setup: Create Temp Views for Join DataFrames

In [16]:
# High Cardinality DataFrame
high_cardinality_df.createOrReplaceTempView("high_card")
# Low Cardinality DataFrame
low_cardinality_df.createOrReplaceTempView("low_card")

#### 0. CSV

In [17]:
csv_schema = """
    user_id BIGINT,
    product_id BIGINT,
    price INT,
    day_of_year LONG,
    product_name STRING,
    transaction_type STRING
"""
start_time = time.time()

df_csv = spark.read.schema(csv_schema).csv("../data/csv/main_table")

# Alias DataFrames
df_csv_alias = df_csv.alias("a")
high_cardinality_df_alias = high_cardinality_df.alias("b")

# Window specification
windowSpec_csv = Window.partitionBy("a.transaction_type").orderBy("total_transactions")

# Perform join, filter and aggregation
df_csv_alias.join(
    high_cardinality_df_alias,
    (df_csv_alias["user_id"] == high_cardinality_df_alias["transaction_id"]) &
    (high_cardinality_df_alias["store_id"] >= 200) &
    (high_cardinality_df_alias["store_id"] <= 300)
).groupBy(
    "a.product_name",
    "a.transaction_type"
).agg(
    F.min("a.price").alias("min_price"),
    F.max("a.price").alias("max_price"),
    F.avg("a.price").alias("avg_price"),
    F.sum("b.discount").alias("total_discount"),
    F.count("a.product_id").alias("total_transactions")
).withColumn(
    "running_total",
    F.sum("total_transactions").over(windowSpec_csv)
).withColumn(
    "running_avg",
    F.avg("avg_price").over(windowSpec_csv)
).show(n=5, truncate=False)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Time elapsed: {elapsed_time} seconds")

+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|product_name|transaction_type|min_price|max_price|avg_price        |total_discount|total_transactions|running_total|running_avg      |
+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|Accessories |High Value      |80       |99       |89.4948312319867 |39961734      |807736            |807736       |89.4948312319867 |
|Tablet      |High Value      |80       |99       |89.49627143737823|39996605      |807818            |1615554      |89.49555133468246|
|Mobile      |High Value      |80       |99       |89.50407044462351|39991525      |807897            |2423451      |89.49839103799614|
|Laptop      |High Value      |80       |99       |89.49057357583195|40020453      |808472            |3231923      |89.49643667245509|
|Headphones  |High Value      |80       |99     

#### 1. Parquet

In [18]:
start_time = time.time()

df_parquet = spark.read.parquet("../data/parquet/main_table")

# Alias DataFrames
df_parquet_alias = df_parquet.alias("a")
high_cardinality_df_alias = high_cardinality_df.alias("b")

# Window specification
windowSpec_pq = Window.partitionBy("a.transaction_type").orderBy("total_transactions")

# Perform join, filter and aggregation
df_parquet_alias.join(
    high_cardinality_df_alias,
    (df_parquet_alias["user_id"] == high_cardinality_df_alias["transaction_id"]) &
    (high_cardinality_df_alias["store_id"] >= 200) &
    (high_cardinality_df_alias["store_id"] <= 300)
).groupBy(
    "a.product_name",
    "a.transaction_type"
).agg(
    F.min("a.price").alias("min_price"),
    F.max("a.price").alias("max_price"),
    F.avg("a.price").alias("avg_price"),
    F.sum("b.discount").alias("total_discount"),
    F.count("a.product_id").alias("total_transactions")
).withColumn(
    "running_total",
    F.sum("total_transactions").over(windowSpec_pq)
).withColumn(
    "running_avg",
    F.avg("avg_price").over(windowSpec_pq)
).show(n=5, truncate=False)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time elapsed: {elapsed_time} seconds")

+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|product_name|transaction_type|min_price|max_price|avg_price        |total_discount|total_transactions|running_total|running_avg      |
+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|Accessories |High Value      |80       |99       |89.4948312319867 |39961734      |807736            |807736       |89.4948312319867 |
|Tablet      |High Value      |80       |99       |89.49627143737823|39996605      |807818            |1615554      |89.49555133468246|
|Mobile      |High Value      |80       |99       |89.50407044462351|39991525      |807897            |2423451      |89.49839103799614|
|Laptop      |High Value      |80       |99       |89.49057357583195|40020453      |808472            |3231923      |89.49643667245509|
|Headphones  |High Value      |80       |99     

#### 2. Hive Metastore Table (Bucketed)

In [19]:
high_cardinality_metastore_bucketed = run_sql(
    """
    WITH AggregatedData AS (
        SELECT 
        a.product_name,
        a.transaction_type,
        MIN(a.price) AS min_price,
        MAX(a.price) AS max_price,
        AVG(a.price) AS avg_price,
        SUM(b.discount) AS total_discount,
        COUNT(a.product_id) AS total_transactions
        FROM default_db.default_bucketed_table a
        JOIN high_card b
        ON a.user_id = b.transaction_id
        AND b.store_id >= 200
        AND b.store_id <= 300
        GROUP BY a.product_name, a.transaction_type
    ),
    WindowedData AS (
    SELECT *,
        SUM(total_transactions) OVER (PARTITION BY transaction_type ORDER BY total_transactions) AS running_total,
        AVG(avg_price) OVER (PARTITION BY transaction_type ORDER BY total_transactions) AS running_avg
        FROM AggregatedData
    )
    SELECT * FROM WindowedData
    """,
    show=True
)

+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|product_name|transaction_type|min_price|max_price|avg_price        |total_discount|total_transactions|running_total|running_avg      |
+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|Accessories |High Value      |80       |99       |89.4948312319867 |39961734      |807736            |807736       |89.4948312319867 |
|Tablet      |High Value      |80       |99       |89.49627143737823|39996605      |807818            |1615554      |89.49555133468246|
|Mobile      |High Value      |80       |99       |89.50407044462351|39991525      |807897            |2423451      |89.49839103799614|
|Laptop      |High Value      |80       |99       |89.49057357583195|40020453      |808472            |3231923      |89.49643667245509|
|Headphones  |High Value      |80       |99     

#### 3. Hive Metastore Table (Partitioned)

In [20]:
high_cardinality_metastore_partitioned = run_sql(
    """
    WITH AggregatedData AS (
        SELECT 
        a.product_name,
        a.transaction_type,
        MIN(a.price) AS min_price,
        MAX(a.price) AS max_price,
        AVG(a.price) AS avg_price,
        SUM(b.discount) AS total_discount,
        COUNT(a.product_id) AS total_transactions
        FROM default_db.default_partitioned_table a
        JOIN high_card b
        ON a.user_id = b.transaction_id
        AND b.store_id >= 200
        AND b.store_id <= 300
        GROUP BY a.product_name, a.transaction_type
    ),
    WindowedData AS (
    SELECT *,
        SUM(total_transactions) OVER (PARTITION BY transaction_type ORDER BY total_transactions) AS running_total,
        AVG(avg_price) OVER (PARTITION BY transaction_type ORDER BY total_transactions) AS running_avg
        FROM AggregatedData
    )
    SELECT * FROM WindowedData
    """,
    show=True
)

+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|product_name|transaction_type|min_price|max_price|avg_price        |total_discount|total_transactions|running_total|running_avg      |
+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|Accessories |High Value      |80       |99       |89.4948312319867 |39961734      |807736            |807736       |89.4948312319867 |
|Tablet      |High Value      |80       |99       |89.49627143737823|39996605      |807818            |1615554      |89.49555133468246|
|Mobile      |High Value      |80       |99       |89.50407044462351|39991525      |807897            |2423451      |89.49839103799614|
|Laptop      |High Value      |80       |99       |89.49057357583195|40020453      |808472            |3231923      |89.49643667245509|
|Headphones  |High Value      |80       |99     

#### 4. Delta Table

In [21]:
high_cardinality_delta = run_sql(
    """
    WITH AggregatedData AS (
        SELECT 
        a.product_name,
        a.transaction_type,
        MIN(a.price) AS min_price,
        MAX(a.price) AS max_price,
        AVG(a.price) AS avg_price,
        SUM(b.discount) AS total_discount,
        COUNT(a.product_id) AS total_transactions
        FROM delta_db.main_table a
        JOIN high_card b
        ON a.user_id = b.transaction_id
        AND b.store_id >= 200
        AND b.store_id <= 300
        GROUP BY a.product_name, a.transaction_type
    ),
    WindowedData AS (
    SELECT *,
        SUM(total_transactions) OVER (PARTITION BY transaction_type ORDER BY total_transactions) AS running_total,
        AVG(avg_price) OVER (PARTITION BY transaction_type ORDER BY total_transactions) AS running_avg
        FROM AggregatedData
    )
    SELECT * FROM WindowedData
    """,
    show=True
)

+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|product_name|transaction_type|min_price|max_price|avg_price        |total_discount|total_transactions|running_total|running_avg      |
+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|Accessories |High Value      |80       |99       |89.4948312319867 |39961734      |807736            |807736       |89.4948312319867 |
|Tablet      |High Value      |80       |99       |89.49627143737823|39996605      |807818            |1615554      |89.49555133468246|
|Mobile      |High Value      |80       |99       |89.50407044462351|39991525      |807897            |2423451      |89.49839103799614|
|Laptop      |High Value      |80       |99       |89.49057357583195|40020453      |808472            |3231923      |89.49643667245509|
|Headphones  |High Value      |80       |99     

#### 5. Iceberg Table

In [22]:
high_cardinality_iceberg = run_sql(
    """
    WITH AggregatedData AS (
        SELECT 
        a.product_name,
        a.transaction_type,
        MIN(a.price) AS min_price,
        MAX(a.price) AS max_price,
        AVG(a.price) AS avg_price,
        SUM(b.discount) AS total_discount,
        COUNT(a.product_id) AS total_transactions
        FROM iceberg_db.main_table a
        JOIN high_card b
        ON a.user_id = b.transaction_id
        AND b.store_id >= 200
        AND b.store_id <= 300
        GROUP BY a.product_name, a.transaction_type
    ),
    WindowedData AS (
    SELECT *,
        SUM(total_transactions) OVER (PARTITION BY transaction_type ORDER BY total_transactions) AS running_total,
        AVG(avg_price) OVER (PARTITION BY transaction_type ORDER BY total_transactions) AS running_avg
        FROM AggregatedData
    )
    SELECT * FROM WindowedData
    """,
    show=True
)

+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|product_name|transaction_type|min_price|max_price|avg_price        |total_discount|total_transactions|running_total|running_avg      |
+------------+----------------+---------+---------+-----------------+--------------+------------------+-------------+-----------------+
|Accessories |High Value      |80       |99       |89.4948312319867 |39961734      |807736            |807736       |89.4948312319867 |
|Tablet      |High Value      |80       |99       |89.49627143737823|39996605      |807818            |1615554      |89.49555133468246|
|Mobile      |High Value      |80       |99       |89.50407044462351|39991525      |807897            |2423451      |89.49839103799614|
|Laptop      |High Value      |80       |99       |89.49057357583195|40020453      |808472            |3231923      |89.49643667245509|
|Headphones  |High Value      |80       |99     

### Low Cardinality Join Comparison
---

#### 0. CSV

In [23]:
start_time = time.time()

df_csv.join(low_cardinality_df, df_csv.product_id == low_cardinality_df.store_id) \
    .where("store_region = 1") \
    .count()

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Time elapsed: {elapsed_time} seconds")

Time elapsed: 32.20572590827942 seconds


#### 1. Parquet

In [24]:
start_time = time.time()

df_parquet.join(low_cardinality_df, df_parquet.product_id == low_cardinality_df.store_id) \
    .where("day_of_year > 200 AND day_of_year < 365") \
    .count()

end_time = time.time()
elapsed_time = end_time - start_time

print(f"Time elapsed: {elapsed_time} seconds")

Time elapsed: 2.8671212196350098 seconds


#### 2. Hive Metastore Table (Bucketed)

In [25]:
low_cardinality_bucketed = run_sql(
    """
    SELECT COUNT(*)
    FROM default_db.default_bucketed_table a
    JOIN low_card b
    ON a.product_id = b.store_id
    WHERE day_of_year > 200 AND day_of_year < 365
    """,
    show=True
)

+--------+
|count(1)|
+--------+
|44931427|
+--------+

Time elapsed: 2.1369290351867676 seconds


#### 3. Hive Metastore Table (Partitioned)

In [26]:
low_cardinality_partitioned = run_sql(
    """
    SELECT COUNT(*)
    FROM default_db.default_partitioned_table a
    JOIN low_card b
    ON a.product_id = b.store_id
    WHERE day_of_year > 200 AND day_of_year < 365
    """,
    show=True
)

+--------+
|count(1)|
+--------+
|44931427|
+--------+

Time elapsed: 1.7597320079803467 seconds


#### 4. Delta Table

In [27]:
low_cardinality_delta = run_sql(
    """
    SELECT COUNT(*)
    FROM delta_db.main_table a
    JOIN low_card b
    ON a.product_id = b.store_id
    WHERE day_of_year > 200 AND day_of_year < 365
    """,
    show=True
)

+--------+
|count(1)|
+--------+
|44931427|
+--------+

Time elapsed: 1.3927130699157715 seconds


#### 5. Iceberg

In [28]:
low_cardinality_iceberg = run_sql(
    """
    SELECT COUNT(*)
    FROM iceberg_db.main_table a
    JOIN low_card b
    ON a.product_id = b.store_id
    WHERE day_of_year > 200 AND day_of_year < 365
    """,
    show=True
)

+--------+
|count(1)|
+--------+
|44931427|
+--------+

Time elapsed: 3.0303919315338135 seconds


In [29]:
spark.stop()

---
**E.O.D**